In [1]:
import pandas as pd
import numpy as np

In [2]:
# this contains the dataset to which we are going to add the target variable

definitive_dataset = pd.read_csv('../../data/definitive_dataset.csv')

In [3]:
# this contains the not differentiated dataset, from which we are going to build the target variable from

data = pd.read_csv('../../data/backtest_prices.csv')
data['date'] = pd.to_datetime(data['date'])

# keep only the relevant columns
data = data[['date', 'permno', 'adj_prc_w_dividend']]
data.rename(columns={'adj_prc_w_dividend': 'prc_adj'}, inplace=True)

# keep only the date and not the time of the day
data['date'] = data['date'].dt.date

# add a column of nans
data['target'] = np.nan

In [4]:
subset_date = '2007-12-31'
subset_date = pd.to_datetime(subset_date).date()

# subset the data after the date
data_subset = data[data['date'] > subset_date]

In [5]:
data_subset['date']

0          2008-12-09
400        2008-01-02
401        2008-01-03
402        2008-01-04
403        2008-01-07
              ...    
2675492    2016-08-31
2675493    2016-09-01
2675494    2016-09-02
2675495    2016-09-06
2675496    2016-09-06
Name: date, Length: 2405597, dtype: object

In [6]:
# find the unique values of the date column
unique_dates = data_subset['date'].unique()

unique_dates = pd.to_datetime(unique_dates)

print(unique_dates)

n_dates = len(unique_dates)
print(n_dates)

DatetimeIndex(['2008-12-09', '2008-01-02', '2008-01-03', '2008-01-04',
               '2008-01-07', '2008-01-08', '2008-01-09', '2008-01-10',
               '2008-01-11', '2008-01-14',
               ...
               '2021-12-17', '2021-12-20', '2021-12-21', '2021-12-22',
               '2021-12-23', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', length=3526, freq=None)
3526


In [7]:
# add a column to the data_subset that contains the date and the permno
data_subset['date_permno'] = data_subset['date'].astype(str) + '_' + data_subset['permno'].astype(str)

C:\Users\Asus\AppData\Local\Temp\ipykernel_21436\2521118835.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset['date_permno'] = data_subset['date'].astype(str) + '_' + data_subset['permno'].astype(str)


In [8]:
# drop the rows which have NaN values in the date
data_subset = data_subset.dropna(subset=['date'])

In [9]:
lagged_data = pd.DataFrame(columns=['date_permno', 'lagged_price'])

# loop through the unique dates
for i in range(n_dates):
    # get the date
    date = unique_dates[i].date()

    #print('Date:', date)

    # get the return date
    return_date = date + pd.DateOffset(days=90)
    return_date = return_date.date()

    # find the return date if it wasn't a trading day going back 1 day at a time
    while return_date not in unique_dates:
        return_date = return_date - pd.DateOffset(days=1)
    
    return_date = return_date.date()
    
    # find the indexes the return date
    return_date_mask = data_subset['date'] == return_date

    # get the lagged price
    lagged_price = data_subset[return_date_mask]['prc_adj'].values

    # get the permno at the return date
    lagged_permnos = data_subset[return_date_mask]['permno'].copy()

    # find the lagged date_permno
    date_permnos = str(date) + '_' + lagged_permnos.astype(str)

    # add to the lagged_data dataframe
    lagged_data = pd.concat([lagged_data, pd.DataFrame({'date_permno': date_permnos.values, 'lagged_price': lagged_price})])

    # print the progress deleting the previous line
    if i % 20 == 0:
        print(f'{i+1}/{n_dates} dates processed', end='\r', flush=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_21436\1530229719.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lagged_data = pd.concat([lagged_data, pd.DataFrame({'date_permno': date_permnos.values, 'lagged_price': lagged_price})])


In [10]:
# merge the lagged data with the data_subset on the date_permno
data_merged = data_subset.merge(lagged_data, on='date_permno', how='left')

In [11]:
data_merged.head()

,date,permno,prc_adj,target,date_permno,lagged_price
0,2008-12-09,10145,26.417659,NaN,2008-12-09_10145,22.523726
1,2008-01-02,10137,62.950000,NaN,2008-01-02_10137,50.500000
2,2008-01-03,10137,63.470000,NaN,2008-01-03_10137,51.840000
3,2008-01-04,10137,62.640000,NaN,2008-01-04_10137,52.350000
4,2008-01-07,10137,63.060000,NaN,2008-01-07_10137,52.810000


In [12]:
# calculate the difference between the lagged price and the price
data_merged['return'] = data_merged['lagged_price'] - data_merged['prc_adj']

# set the target to 1 if the return is greater than 0, to 0 if the return is less than 0
data_merged['target'] = np.where(data_merged['return'] > 0, 1, data_merged['target'])
data_merged['target'] = np.where(data_merged['return'] <= 0, 0, data_merged['target'])

In [13]:
data_merged[data_merged['target'].notna()]

,date,permno,prc_adj,target,date_permno,lagged_price,return
0,2008-12-09,10145,26.417659,0.0,2008-12-09_10145,22.523726,-3.893932
1,2008-01-02,10137,62.950000,0.0,2008-01-02_10137,50.500000,-12.450000
2,2008-01-03,10137,63.470000,0.0,2008-01-03_10137,51.840000,-11.630000
3,2008-01-04,10137,62.640000,0.0,2008-01-04_10137,52.350000,-10.290000
4,2008-01-07,10137,63.060000,0.0,2008-01-07_10137,52.810000,-10.250000
...,...,...,...,...,...,...,...
2415987,2016-06-06,10147,27.970000,1.0,2016-06-06_10147,28.770000,0.800000
2415988,2016-06-07,10147,28.010000,1.0,2016-06-07_10147,28.770000,0.760000
2415989,2016-06-08,10147,27.940000,1.0,2016-06-08_10147,28.770000,0.830000
2415990,2016-06-09,10147,27.870000,1.0,2016-06-09_10147,29.050000,1.180000


In [14]:
# remove printing limits
pd.set_option('display.max_columns', None)

# select n random rows
n = 100
columns_to_print = ['permno', 'date', 'prc_adj', 'lagged_price', 'return', 'target']

# print the random rows
print(data_merged[columns_to_print].sample(n))

         permno        date     prc_adj  lagged_price     return  target
11225     10696  2019-02-01   82.920000     85.930000   3.010000     1.0
2350158   92322  2011-04-29   53.190000     62.080000   8.890000     1.0
1099981   61516  2010-12-08   98.750000    115.940000  17.190000     1.0
1089386   59248  2021-04-26   52.940000     49.700000  -3.240000     0.0
2060187   88860  2018-01-03  231.200000    242.690000  11.490000     1.0
...         ...         ...         ...           ...        ...     ...
1200558   66800  2016-04-11   53.410000     53.010000  -0.400000     0.0
1244684   75034  2020-06-10   17.260000     14.530000  -2.730000     0.0
864293    47466  2020-04-23   30.190000     45.640000  15.450000     1.0
2141925   89626  2009-08-27   55.579998     64.446002   8.866004     1.0
364966    17005  2021-05-18   88.610000     84.360000  -4.250000     0.0

[100 rows x 6 columns]


In [15]:
# drop the date_permno column
data_merged.drop('date_permno', axis=1, inplace=True)

In [16]:
# when return = 0, target = 0
data_merged['target'] = np.where(data_merged['return'] == 0, 0, data_merged['target'])

# drop lagged_price and return columns
data_merged.drop(['lagged_price', 'return'], axis=1, inplace=True)

In [17]:
data_merged[data_merged['target'].isna()]

,date,permno,prc_adj,target
738,2010-12-01,10137,23.01,NaN
739,2010-12-02,10137,23.28,NaN
740,2010-12-03,10137,23.24,NaN
741,2010-12-06,10137,23.15,NaN
742,2010-12-07,10137,23.16,NaN
...,...,...,...,...
2416049,2016-08-31,10147,28.99,NaN
2416050,2016-09-01,10147,28.86,NaN
2416051,2016-09-02,10147,28.77,NaN
2416052,2016-09-06,10147,29.05,NaN


In [20]:
definitive_dataset.columns

Index(['date', 'permno', 'stat_divyeld', 'fed_funds_adj_close',
       'fed_funds_volume', 'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil',
       'pe_exi', 'pe_inc', 'ps', 'pcf', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm',
       'cfm', 'roa', 'roe', 'roce', 'aftret_eq', 'aftret_invcapx',
       'aftret_equity', 'GProf', 'equity_invcap', 'debt_invcap',
       'totdebt_invcap', 'capital_ratio', 'cash_lt', 'debt_at', 'debt_ebitda',
       'short_debt', 'lt_debt', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be',
       'debt_assets', 'debt_capital', 'de_ratio', 'at_turn', 'rect_turn',
       'pay_turn', 'sale_invcap', 'sale_equity', 'rd_sale', 'adv_sale',
       'staff_sale', 'accrual', 'ptb', 'divyield', 'vol', 'ret', 'retx',
       'mktcap', 'naics_processed', 'ret_industry_tot',
       'ret_industry_relative', '3_month_return', '12_month_return',
       'MACD_index', 'rsi', 'prc_adj', 'prc'],
      dtype='object')

In [21]:
# merge data_merged[['date', 'permno', 'target']] with definitive_dataset on date and permno
definitive_dataset['date'] = pd.to_datetime(definitive_dataset['date']).dt.date
data_merged['date'] = pd.to_datetime(data_merged['date']).dt.date
#definitive_dataset_2 = definitive_dataset.drop(columns=['target'])
definitive_dataset_2 = definitive_dataset.merge(data_merged[['date', 'permno', 'target']], on=['date', 'permno'], how='inner')

In [24]:
definitive_dataset_2.columns

Index(['date', 'permno', 'stat_divyeld', 'fed_funds_adj_close',
       'fed_funds_volume', 'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil',
       'pe_exi', 'pe_inc', 'ps', 'pcf', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm',
       'cfm', 'roa', 'roe', 'roce', 'aftret_eq', 'aftret_invcapx',
       'aftret_equity', 'GProf', 'equity_invcap', 'debt_invcap',
       'totdebt_invcap', 'capital_ratio', 'cash_lt', 'debt_at', 'debt_ebitda',
       'short_debt', 'lt_debt', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be',
       'debt_assets', 'debt_capital', 'de_ratio', 'at_turn', 'rect_turn',
       'pay_turn', 'sale_invcap', 'sale_equity', 'rd_sale', 'adv_sale',
       'staff_sale', 'accrual', 'ptb', 'divyield', 'vol', 'ret', 'retx',
       'mktcap', 'naics_processed', 'ret_industry_tot',
       'ret_industry_relative', '3_month_return', '12_month_return',
       'MACD_index', 'rsi', 'prc_adj', 'prc', 'target'],
      dtype='object')

In [23]:
# save the data to a csv file
definitive_dataset_2.to_csv('../../data/definitive_dataset.csv', index=False)

In [32]:
data_merged.to_csv('../../data/non_diff_adjusted_price_data.csv', index=False)